### 输出解析器 Output Parsers

#### PydanticOutputParser

In [1]:
import os
os.environ["OPENAI_API_KEY"] = "sk-xxx"
os.environ["OPENAI_API_BASE"] = "https://api.chatanywhere.tech/v1"

In [3]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import OpenAI

model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)


# 定义所需的数据结构。
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # 使用 Pydantic 添加自定义验证逻辑。
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


# 设置一个解析器 + 将指令注入到提示模板中。
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 用于提示语言模型填充数据结构的查询。
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Tell me a joke."})
formatRes = parser.invoke(output)
print(output)
print('format', output)


{
    "setup": "Why did the tomato turn red?",
    "punchline": "Because it saw the salad dressing!"
}
format 
{
    "setup": "Why did the tomato turn red?",
    "punchline": "Because it saw the salad dressing!"
}


#### 列表解析 

将结果输出成列表形式

In [24]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4-turbo")

output_parser = CommaSeparatedListOutputParser()


template = "Generate a list of 5 {text}.\n\n{format_instructions}"


chat_prompt = ChatPromptTemplate.from_template(template)
chat_prompt = chat_prompt.partial(format_instructions=output_parser.get_format_instructions())
chain = chat_prompt | chat_model

list_output = chain.invoke({"text": "colors"})
print(list_output)
format_output = output_parser.invoke(list_output.content)
print(format_output)


content='Blue, Red, Green, Yellow, Purple' response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 35, 'total_tokens': 44}, 'model_name': 'gpt-4-turbo', 'system_fingerprint': 'fp_ea6eb70039', 'finish_reason': 'stop', 'logprobs': None} id='run-07aafa89-06c0-4c95-900a-ccd0b56de338-0'
['Blue', 'Red', 'Green', 'Yellow', 'Purple']
